# 코드 실행 전 CSV 변환, 파일 내 이상한 문구 수정 후 돌릴 것

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np


import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.graphics.mosaicplot import mosaic
from scipy import stats as spst
import statsmodels.api as sm

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


# 1. 분석할 데이터 경로 및 파일 설정

In [3]:
cd /content/drive/MyDrive/test/

/content/drive/MyDrive/test


In [4]:
path = "/content/drive/MyDrive/test/phone_1.csv"
data = pd.read_csv(path)

In [5]:
len(data)

21858

In [6]:
# 불러온 파일의 결측지를 NN이라는 문자로 채우기

data.fillna("NaN",inplace=True)

# 2. 필요한 컬럼만 뽑아 복사본 만들기

In [49]:
colums = ["index", "respondentID2","respondentID","externalID","Country","CX2_1ab_EK","DR6_1_EK","DR10_1_Etc_EK"]

copy = data[colums].copy()

# 3. 유사한 답변을 한 colum들을 뽑아 row 삭제

- index만 추출

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
# 유사도가 높은 colum을 삭제할 값 저장 

col_delete_data = {
    "respondentID2": [],
    "respondentID": [],
    "externalID" : [],
    "Country": [],
    "CX2_1ab_EK" : [],
    "DR6_1_EK" : [],
    "DR10_1_Etc_EK" : [],
}

# 유사도가 높은 row을 삭제할 값 저장 
row_delete_data = {
    "respondentID2": [],
    "respondentID": [],
    "externalID" : [],
    "Country": [],
    "CX2_1ab_EK" : [],
    "DR6_1_EK" : [],
    "DR10_1_Etc_EK" : [],
}

# 유사도가 높은 colum, row 삭제할 값 저장 
delete_data = {
    "respondentID2": [],
    "respondentID": [],
    "externalID" : [],
    "Country": [],
    "CX2_1ab_EK" : [],
    "DR6_1_EK" : [],
    "DR10_1_Etc_EK" : [],
}

A = copy["CX2_1ab_EK"].values
A = A.tolist()
B = copy["DR6_1_EK"].values
B = B.tolist()
C = copy["DR10_1_Etc_EK"].values
C = C.tolist()

# 최종적으로 활용할 배열 --> 삭제할 인덱스 저장
col_array = []

In [9]:
for i in range(len(copy)):
  mydoclist = [A[i],B[i],C[i]]
  
  # 유사도 값 추출과 array 생성
  tfidf_vectorizer = TfidfVectorizer(min_df=1)
  tfidf_matrix = tfidf_vectorizer.fit_transform(mydoclist)
  document_distances = (tfidf_matrix * tfidf_matrix.T)

  # avg = document_distances.toarray()[::1][3][:3].sum() / 3  3개의 유사도의 총합의 평균 --> 한쪽이 유사도가 0.9가 나와도 다른게 0.2, 0.1 나올때 평균이 너무 낮아져서 유사한 답변이라고 취급 안하는 문제 발생
  
  
  # nan값인 경우는 그냥 패스 해서 중복 삭제하는 것을 방지
  if "NaN" in mydoclist and (document_distances.toarray()[0][2] == 1 or document_distances.toarray()[1][2] == 1 or document_distances.toarray()[0][1] == 1) :
    continue

  #  각각의 유사도가 하나라도 기준 유사도(0.55)을 넘으면 같다고 보고 삭제하는 조건문
  if  document_distances.toarray()[0][2] >=0.55 or document_distances.toarray()[1][2]>=0.55 or document_distances.toarray()[0][1] >= 0.55 :
    # 삭제할 index 찾기
    idx = copy[copy["index"] == i ].index
    col_array.append(idx[0])

In [10]:
len(col_array)

583

# 3. 유사한 답변을 한 row들을 뽑아 row 삭제 0707 기준 : 시간 복잡도 개선
- index만 추출

In [11]:
pip install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.4 MB 1.2 MB/s 
     |████████████████████████████████| 453 kB 48.2 MB/s 


In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

import nltk
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [13]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
from konlpy.tag import Okt

stop_words_list = stopwords.words('english')
stop_words_list.extend(["Ah","ah"])
tfidf = TfidfVectorizer(stop_words=stop_words_list) # stop_words를 english를 할 경우 the, a와 같은 의미 없는 단어는 스킵

In [14]:
A = copy["CX2_1ab_EK"].values
A = A.tolist()
B = copy["DR6_1_EK"].values
B = B.tolist()
C = copy["DR10_1_Etc_EK"].values
C = C.tolist()

key = "setence"
row_text = {}
row_text[key]=[]

for i in range(len(copy)):
  A[i] = A[i].replace("NaN"," ")
  B[i] = B[i].replace("NaN"," ")
  C[i] = C[i].replace("NaN"," ")
  row_text[key].append(str(A[i]) + str(B[i]) + str(C[i]))

In [15]:
row_text =pd.DataFrame(row_text)

In [16]:
row_text.head()

,setence
0,처음에 점원이 부족해서 상담까지 대기가 너무 길었다. 이후 계산 등에도 시간이 많이...
1,온라인으로 검색하고 정보를 얻는것도 좋지만 온라인보다 오프라인으로 보는 것이 더 확...
2,친절하고 화면 배색도 좋아요 그리고 무엇보다 친절한 설명이있어서 좋아요 물건검색 하...
3,접근성 개선이 필요합니다 왜냐하면 일렉트로마트까지 가는데 30분이 걸리기 때문입니다...
4,"실제 이용자들의 리뷰와 홍보용 기사들을 분리해서 파악할 수 있었으며, 이로 인해 전..."


In [89]:
junk = ["survey"]
idx =0
cnt = 0
for trash in row_text["setence"]:
  if trash in junk:
    print(idx, trash)
    cnt+=1
  idx+=1
print(cnt)

0


In [17]:
# row 끼리의 코사인 유사도 메트릭스 만들기
tfidf_matrix = tfidf.fit_transform(row_text["setence"])
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
cosine_sim

array([[1.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 1.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.09310174,
        0.08149182],
       [0.        , 0.        , 0.        , ..., 0.09310174, 1.        ,
        0.06722259],
       [0.        , 0.        , 0.        , ..., 0.08149182, 0.06722259,
        1.        ]])

In [18]:
cosine_sim.shape

(21858, 21858)

In [19]:
# 기준 유사도를 넘는 index를 row_dic에 저장
row_dic = {}
for key in copy["index"]:
  row_dic[key] = []
  
key = 0
for idx in cosine_sim:
  for j in range(len(idx)):
    if idx[j] >= 0.8 and idx[j]<=1 and key !=j : # 기준 의논 
      if j not in row_dic[key]:
        row_dic[key].append(j)
  if key%1000 == 0:
    print(key, row_dic[key])
  key+=1

0 []
1000 []
2000 []
3000 []
4000 []
5000 []
6000 []
7000 []
8000 []
9000 []
10000 []
11000 []
12000 []
13000 []
14000 []
15000 []
16000 []
17000 []
18000 []
19000 []
20000 []
21000 []


In [ ]:
for key in row_dic:
  if len(row_dic[key]) >1:
    print(key, row_dic[key])

3203 [2923, 4209]
4617 [2708, 5045]
4631 [10374, 11915]
5902 [15121, 15454]
6244 [7409, 7656, 11893]
6309 [7280, 7413, 7604]
7409 [6244, 11893]
7413 [6309, 7604]
7604 [6309, 7413]
10013 [11269, 11981]
10241 [11309, 11311]
10330 [10866, 10986]
10374 [4631, 11915]
10436 [10963, 11748]
10592 [11189, 11894]
10753 [12112, 12320]
10963 [10436, 11748, 12218]
11189 [10592, 11894]
11269 [10013, 17432]
11748 [10436, 10963]
11778 [10909, 11956, 12005]
11872 [11287, 11404]
11893 [6244, 7409, 11981]
11894 [10592, 11189]
11915 [4631, 10374]
11956 [11778, 12005]
11981 [10013, 11893, 12036, 15536, 15875]
12005 [11778, 11956]
12112 [10753, 12320, 15732]
12238 [11410, 11489, 12351]
12320 [10753, 12112]
12415 [11275, 12106]
13516 [13522, 13523]
13522 [13516, 13523, 13640]
13523 [13516, 13522, 13640]
13600 [13882, 13904]
13640 [13522, 13523]
13807 [13809, 13831, 13858, 13863]
13809 [13807, 13831, 13858, 13863]
13831 [13807, 13809, 13858]
13858 [13807, 13809, 13831, 13863]
13863 [13807, 13809, 13858]
13882

In [20]:
# row_dic key와 유사한 위치의 인덱스들에서 인덱스만 따로 저장하는 배열 생성(key값은 미 포함)
row_array = []
for k in row_dic:
  if len(row_dic[k]) >0:
    for value in row_dic[k]:
      if value not in row_array:
        row_array.append(value)

In [21]:
len(row_array)

129

In [22]:
# col + row index만 모아 놓은 array
array = []
array.extend(row_array)
array.extend(col_array)
len(array)

712

In [23]:
# 중복 삭제
array = list(set(array))
array = sorted(array)

In [24]:
len(array)

703

In [25]:
# 해당 인덱스에 해당하는 colum들 값 을 col_delete_data에 집어 넣기

for i in col_array:

    idx = copy[copy["index"] == i ].index
    
    col_delete_data["respondentID2"].extend(copy.loc[:,"respondentID2" ][idx].tolist())
    col_delete_data["respondentID"].extend(copy.loc[:,"respondentID" ][idx].tolist())
    col_delete_data["externalID"].extend(copy.loc[:,"externalID" ][idx].tolist())
    col_delete_data["Country"].extend(copy.loc[:,"Country"][idx].tolist())
    col_delete_data["CX2_1ab_EK"].extend(copy.loc[:,"CX2_1ab_EK" ][idx].tolist())
    col_delete_data["DR6_1_EK"].extend(copy.loc[:,"DR6_1_EK"][idx].tolist())
    col_delete_data["DR10_1_Etc_EK"].extend(copy.loc[:,"DR10_1_Etc_EK" ][idx].tolist())

In [26]:
# 해당 인덱스에 해당하는 row들 값 을 row_delete_data에 집어 넣기

for i in row_array:

    idx = copy[copy["index"] == i ].index
    
    row_delete_data["respondentID2"].extend(copy.loc[:,"respondentID2" ][idx].tolist())
    row_delete_data["respondentID"].extend(copy.loc[:,"respondentID" ][idx].tolist())
    row_delete_data["externalID"].extend(copy.loc[:,"externalID" ][idx].tolist())
    row_delete_data["Country"].extend(copy.loc[:,"Country"][idx].tolist())
    row_delete_data["CX2_1ab_EK"].extend(copy.loc[:,"CX2_1ab_EK" ][idx].tolist())
    row_delete_data["DR6_1_EK"].extend(copy.loc[:,"DR6_1_EK"][idx].tolist())
    row_delete_data["DR10_1_Etc_EK"].extend(copy.loc[:,"DR10_1_Etc_EK" ][idx].tolist())

In [27]:
# 해당 인덱스에 해당하는 전체 값 을 delete_data에 집어 넣기

for i in array:

    idx = copy[copy["index"] == i ].index
    
    delete_data["respondentID2"].extend(copy.loc[:,"respondentID2" ][idx].tolist())
    delete_data["respondentID"].extend(copy.loc[:,"respondentID" ][idx].tolist())
    delete_data["externalID"].extend(copy.loc[:,"externalID" ][idx].tolist())
    delete_data["Country"].extend(copy.loc[:,"Country"][idx].tolist())
    delete_data["CX2_1ab_EK"].extend(copy.loc[:,"CX2_1ab_EK" ][idx].tolist())
    delete_data["DR6_1_EK"].extend(copy.loc[:,"DR6_1_EK"][idx].tolist())
    delete_data["DR10_1_Etc_EK"].extend(copy.loc[:,"DR10_1_Etc_EK" ][idx].tolist())


    # 해당 index의 로우 삭제하고 copy 업데이트
    copy = copy.drop(idx)

In [ ]:
copy.shape

(21137, 8)

In [ ]:
len(delete_data["respondentID2"]), len(delete_data["respondentID"]), len(delete_data["externalID"]), len(delete_data["Country"]), len(delete_data["CX2_1ab_EK"]), len(delete_data["DR6_1_EK"]), len(delete_data["DR10_1_Etc_EK"])

(721, 721, 721, 721, 721, 721, 721)

In [ ]:
copy

,index,respondentID2,respondentID,externalID,Country,CX2_1ab_EK,DR6_1_EK,DR10_1_Etc_EK
0,0,10000028,28,875228488,1,처음에 점원이 부족해서 상담까지 대기가 너무 길었다. 이후 계산 등에도 시간이 많이...,NaN,NaN
1,1,10000087,87,875241028,1,온라인으로 검색하고 정보를 얻는것도 좋지만 온라인보다 오프라인으로 보는 것이 더 확...,제품 후기가 자세하지 않아 따로 인터넷 검색으로 통해 블로그나 인플루언서의 글을 찾...,과거 갤럭시 노트를 사용하다 이 제품을 사용하게 되어 화면이 작다고 느끼게 된다
2,2,10000093,93,875244234,1,친절하고 화면 배색도 좋아요 그리고 무엇보다 친절한 설명이있어서 좋아요 물건검색 하...,NaN,NaN
3,3,10000096,96,875244483,1,접근성 개선이 필요합니다 왜냐하면 일렉트로마트까지 가는데 30분이 걸리기 때문입니다...,딱히 모르겠습니다. 죄송합니다,NaN
4,4,10000112,112,875251428,1,"실제 이용자들의 리뷰와 홍보용 기사들을 분리해서 파악할 수 있었으며, 이로 인해 전...",네이버의 문제라기 보다는 애플 제품의 프로모션이 약한 탓이다.,현재로서는 특별히 불만족스러운 부분이 없다
...,...,...,...,...,...,...,...,...
21853,21853,150007813,7813,875923853,15,"I have a Vivo phone plan, I really like the se...",I didn't see many promotions despite the price...,I find very little storage and the low quality...
21854,21854,150006549,6549,875924519,15,Because some information is given too quickly.,Some points can be improved,Satisfied with the speed and ease of handling.
21855,21855,150007062,7062,875958794,15,"The store has a great service, the products ar...",The store is located far from my house and I d...,"It's a good device, great screen quality, good..."
21856,21856,150017121,17121,880589033,15,I think to receive a 10 you need better servic...,The availability of products and models varies...,I have been using the Apple device for a long ...


# 6. 가공한 데이터 저장

In [29]:
import pandas as pd

# col끼리 유사한 데이터끼리 삭제한 데이터
col_df =pd.DataFrame(col_delete_data)
col_df.to_csv("col_delet_data.csv",columns=["respondentID2","respondentID","externalID","Country","CX2_1ab_EK","DR6_1_EK","DR10_1_Etc_EK"],index=True,encoding='utf-8-sig')

# row끼리 유사한 데이터끼리 삭제한 데이터
row_df =pd.DataFrame(row_delete_data)
row_df.to_csv("row_delet_data.csv",columns=["respondentID2","respondentID","externalID","Country","CX2_1ab_EK","DR6_1_EK","DR10_1_Etc_EK"],index=True,encoding='utf-8-sig')

# 전체 중복 제거한 데잍터만 남겨서 따로 저장
data_df = pd.DataFrame(delete_data)
data_df.to_csv("delet_data.csv",columns=["respondentID2","respondentID","externalID","Country","CX2_1ab_EK","DR6_1_EK","DR10_1_Etc_EK"],index=True,encoding='utf-8-sig')


# 중복 제거 후 남은 데이터 저장
copy.to_csv("test_finish.csv",columns=["respondentID2","respondentID","externalID","Country","CX2_1ab_EK","DR6_1_EK","DR10_1_Etc_EK"],index=True,encoding='utf-8-sig')

# 다음 코드부터는 문법 Test 

In [ ]:
import pandas as pd

a = {'A' : [111, 111, 111, 111, 222, 222], 'B' : ['aaa', 'aaa', 'bbb', 'bbb', 'aaa', 'bbb'], 
'C' : ['zzz', 'zzz', 'zzz', 'yyy', 'zzz', 'yyy']}
df = pd.DataFrame(a)
df

,A,B,C
0,111,aaa,zzz
1,111,aaa,zzz
2,111,bbb,zzz
3,111,bbb,yyy
4,222,aaa,zzz
5,222,bbb,yyy


In [ ]:
df.drop_duplicates()

,A,B,C
0,111,aaa,zzz
2,111,bbb,zzz
3,111,bbb,yyy
4,222,aaa,zzz
5,222,bbb,yyy


In [ ]:
delete_data

In [ ]:
data_df = pd.DataFrame(delete_data)

In [ ]:
data_df

In [ ]:
document_distances.toarray()

In [ ]:
document_distances.toarray()[1]

array([0., 1., 0., 0.])

In [ ]:
document_distances.toarray()[1][3]

0.0

In [ ]:
df = pd.DataFrame({'Num':[1, 2, 2, 2, 2, 3],
                  'Alphabet':['a', 'b', 'b', 'd', 'e', 'f']})
df

,Num,Alphabet
0,1,a
1,2,b
2,2,b
3,2,d
4,2,e
5,3,f


In [ ]:
df.drop_duplicates(subset=None, keep = "first" ,inplace=False, ignore_index=False)

,Num,Alphabet
0,1,a
1,2,b
3,2,d
4,2,e
5,3,f


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
d1 = '사과 바나나 오렌지 바나나 멜론'

d2 = '포도 포도 오렌지 오렌지 수박'

d3 = '사과 사과 바나나 오렌지 수박'

d4 = '포도 사과 멜론 딸기 딸기'

d5 = '딸기 수박 멜론 바나나 오렌지'
documents = [d1, d2, d3, d4, d5]

# TF-IDF 점수 변환 모델 준비
tf_idf_model = TfidfVectorizer().fit(documents)
print(tf_idf_model.vocabulary_.items())
# 단어 이름 리스트 순서대로 반환(0번으로 지정된 토큰부터)
word_id_list = sorted(tf_idf_model.vocabulary_.items(), key=lambda x: x[1], reverse=False)
# print(word_id_list)
word_list = [x[0] for x in word_id_list]

# 용이한 시각화를 위하여 데이터프레임 변환
tf_idf_df = pd.DataFrame(tf_idf_model.transform(documents).toarray(), columns = word_list, index = ['d1', 'd2', 'd3', 'd4', 'd5'])

tf_idf_df

dict_items([('사과', 3), ('바나나', 2), ('오렌지', 5), ('멜론', 1), ('포도', 6), ('수박', 4), ('딸기', 0)])


,딸기,멜론,바나나,사과,수박,오렌지,포도
d1,0.000000,0.386113,0.772225,0.386113,0.000000,0.324810,0.000000
d2,0.000000,0.000000,0.000000,0.000000,0.322149,0.542004,0.776178
d3,0.000000,0.000000,0.386113,0.772225,0.386113,0.324810,0.000000
d4,0.791926,0.328685,0.000000,0.328685,0.000000,0.000000,0.395963
d5,0.530389,0.440271,0.440271,0.000000,0.440271,0.370369,0.000000


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

cos_sim_df = pd.DataFrame(cosine_similarity(tf_idf_df, tf_idf_df), columns = ['d1', 'd2', 'd3', 'd4', 'd5'], index = ['d1', 'd2', 'd3', 'd4', 'd5'])

cos_sim_df

,d1,d2,d3,d4,d5
d1,1.000000,0.176048,0.701834,0.253819,0.630282
d2,0.176048,1.000000,0.300434,0.307338,0.342574
d3,0.701834,0.300434,1.000000,0.253819,0.460288
d4,0.253819,0.307338,0.253819,1.000000,0.564739
d5,0.630282,0.342574,0.460288,0.564739,1.000000


In [54]:
# 

test_colums = ["index"]
n = int(input("몇 개의 컬럼을 받으시겠습니까?\n"))
print(data.columns[1:])
cnt = 0
while cnt != n:
  selection = input()
  if selection in data.columns:
    test_colums.append(selection)
    cnt+=1
  else:
    print("There is not this column")
print(test_colums)

몇 개의 컬럼을 받으시겠습니까?
3
Index(['Product', 'GU1', 'respondentID2', 'respondentID', 'externalID',
       'Country', 'Month', 'SC1', 'AGE#', 'SC8', 'SC9', 'SC13a', 'SC13b',
       'SC14', 'TP12', 'TP13', 'DR2A', 'DR3_1', 'CX2_1ab', 'CX2_1ab_EN',
       'CX2_1ab_EK', 'DR4A_1', 'DR4A_2', 'DR4A_3', 'DR4A_4', 'DR4A_5',
       'DR4A_6', 'DR4A_7', 'DR4A_8', 'DR4A_29', 'DR4A_14', 'DR4A_15',
       'DR4A_16', 'DR4A_17', 'DR4A_18', 'DR4A_19', 'DR4A_20', 'DR4A_21',
       'DR4A_23', 'DR4A_24', 'DR4A_25', 'DR4A_26', 'DR4A_28', 'DR4A_30',
       'DR6_1', 'DR6_1_EN', 'DR6_1_EK', 'DR2B', 'DR3_2', 'CX2_2ab',
       'CX2_2ab_EN', 'CX2_2ab_EK', 'DR4B_1', 'DR4B_2', 'DR4B_3', 'DR4B_4',
       'DR4B_5', 'DR4B_6', 'DR4B_7', 'DR4B_8', 'DR4B_29', 'DR4B_14', 'DR4B_15',
       'DR4B_16', 'DR4B_17', 'DR4B_18', 'DR4B_19', 'DR4B_20', 'DR4B_21',
       'DR4B_23', 'DR4B_24', 'DR4B_25', 'DR4B_26', 'DR4B_28', 'DR4B_30',
       'DR6_2', 'DR6_2_EN', 'DR6_2_EK', 'DR16_1', 'DR16_1_EN', 'DR16_1_EK',
       'DR16_2', 'DR16_2_EN',

In [67]:
a = [[1,2,3],
     [4,5,6],
     [7,8,9],
     [10,11,12]]

In [70]:
print(a[0])
print(a[1][2])
print(len(a))
print(len(a[0]))

[1, 2, 3]
6
4
3


In [77]:
for row in a:
  print(row)
  for i in range(len(row)):
    print(row[i])

[1, 2, 3]
1
2
3
[4, 5, 6]
4
5
6
[7, 8, 9]
7
8
9
[10, 11, 12]
10
11
12
